# <center> WillGraceApartment </center>

![piantina](./img/piantinaGirata.png)
![piantina](./img/Screenshot_1.png)
![piantina](./img/Screenshot_2.png)
![piantina](./img/Screenshot_3.png)
![piantina](./img/Screenshot_4.png)
![piantina](./img/Screenshot_5.png)
![piantina](./img/Screenshot_6.png)
![piantina](./img/Screenshot_7.png)

In [1]:
%run "./My_House_Functions.ipynb"

Evaluating fenvs.py..
...fenvs.py imported in 0.0182754627452 seconds


In [2]:
"""constant var and path"""
# Path
pathExternalWalls = "./lines/external.lines"
pathInternalWalls = "./lines/internal.lines"
pathGrindWalls = "./lines/grind.lines"
pathFloor = "./lines/floor.lines"
pathStairs = "./lines/stairs.lines"
pathStairs2 = "./lines/stairs2.lines"
pathBalconyFloor = "./lines/balconyFloor.lines"
pathDoors = "./lines/doors2.lines"
pathMainEntrance = "./lines/portoneIngresso.lines"
pathWindows = "./lines/windows3.lines"
pathAperture = "./lines/aperture.lines"



# Constant var
externalWallsThickness = 5
internalWallsThickness = 2
grindWallsThickness = 1
grindHeight = 10
houseHeight = 40
windowsHeight = 20
windowsHeightFromFloor = 10
doorsHeight = 25

# Material Orange
#ambientRGBA = [1,0,0,1]
#diffuseRGBA = [0,1,0,1]
#specularRGBA = [0,0,1,0]
#emissionRGBA = [1,0,0,1]
#shininess = 100

wx = [0.6, 8.2, 0.6, 8.2, 0.6, 8.2, 0.6]
wy = 0.3
wz = [0.6, 4.25, 0.6, 4.25, 0.6, 4.25, 0.6, 4.25, 0.6]
wb = [[1,1,1,1,1,1,1],[1,0,0,0,0,0,1],[1,1,1,1,1,1,1],[1,0,1,0,1,0,1],[1,1,1,1,1,1,1],[1,0,1,0,1,0,1],[1,1,1,1,1,1,1],[1,0,1,0,1,0,1],[1,1,1,1,1,1,1]]

In [4]:
VIEW(Apartment(1))

Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000B5A5C00> >